# Matrix Condition Number and Roundoff Error

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WCC-Engineering/ENGR240/blob/main/Class%20Demos%20and%20Activities/Week%204/Matrix_Condition_Number_Demo.ipynb)

## Introduction

In this notebook, we'll explore the concept of **matrix condition number** and how it affects the numerical solution of linear systems of equations. We'll specifically focus on:

1. What makes a matrix "ill-conditioned"
2. How roundoff error accumulates in ill-conditioned systems
3. How to check the condition number of a matrix
4. Basic techniques to improve numerical stability

In [ ]:
# Import the necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy.linalg import cond, solve, norm, inv
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# Configure plot settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.5)

# Set precision for display
np.set_printoptions(precision=8, suppress=True)

## What is the Condition Number?

The **condition number** of a matrix measures how sensitive the solution of a linear system is to small changes or errors in the input data. 

For a matrix $A$, the condition number is defined as:

$$\kappa(A) = ||A|| \cdot ||A^{-1}||$$

where $||\cdot||$ represents a matrix norm (commonly the 2-norm).

- A matrix with a **low condition number** (close to 1) is **well-conditioned**. Small changes in the input result in small changes in the output.
- A matrix with a **high condition number** is **ill-conditioned**. Small changes in the input can cause large changes in the output.
- A matrix with an **infinite condition number** is **singular** (not invertible).

## Example 1: Well-Conditioned Matrix

Let's start with a well-conditioned matrix and see how it behaves when solving a linear system.

In [ ]:
# Create a well-conditioned matrix (a simple diagonal matrix)
A_well = np.array([
    [4.0, 0.0],
    [0.0, 5.0]
])

# True solution we want to recover
x_true = np.array([2.0, 3.0])

# Calculate the right-hand side
b = A_well @ x_true

print("Matrix A (well-conditioned):")
print(A_well)
print("\nRight-hand side b:")
print(b)

# Calculate the condition number
cond_number = cond(A_well)
print(f"\nCondition number: {cond_number:.4f}")

# Solve the system
x_computed = solve(A_well, b)
print("\nComputed solution:")
print(x_computed)

# Calculate the error
error = norm(x_true - x_computed) / norm(x_true)
print(f"\nRelative error: {error:.8e}")

Now, let's introduce some small perturbation to the right-hand side and see how it affects the solution:

In [ ]:
# Add a small perturbation to b (simulating measurement error or roundoff)
delta = 1e-10
b_perturbed = b + delta * np.random.randn(2)

# Solve with the perturbed right-hand side
x_perturbed = solve(A_well, b_perturbed)

# Calculate the relative change in the solution
input_change = norm(b - b_perturbed) / norm(b)
output_change = norm(x_computed - x_perturbed) / norm(x_computed)

print(f"Relative change in input (b): {input_change:.8e}")
print(f"Relative change in output (x): {output_change:.8e}")
print(f"Ratio of output change to input change: {output_change/input_change:.4f}")

For a well-conditioned matrix, the relative change in the solution is similar in magnitude to the relative change in the input data. The ratio is close to the condition number.

## Example 2: Ill-Conditioned Matrix

Now, let's examine an ill-conditioned matrix and observe how it amplifies errors.

In [ ]:
# Create a Hilbert matrix - a classic example of an ill-conditioned matrix
# The Hilbert matrix H has elements H[i,j] = 1/(i+j+1)
def hilbert_matrix(n):
    H = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            H[i, j] = 1.0 / (i + j + 1)
    return H

# Create a larger Hilbert matrix (10x10) to demonstrate more pronounced effects
n = 10
A_ill = hilbert_matrix(n)

# Create a more complex true solution - an alternating pattern
x_true_ill = np.array([(-1)**i * (i+1) for i in range(n)])

# Calculate right-hand side
b_ill = A_ill @ x_true_ill

print("Hilbert matrix (ill-conditioned, showing first 5x5 corner):")
print(A_ill[:5, :5])

# Calculate the condition number
cond_number_ill = cond(A_ill)
print(f"\nCondition number: {cond_number_ill:.4e}")

# Solve the system using double precision
x_computed_ill = solve(A_ill, b_ill)

print("\nTrue solution:")
print(x_true_ill)

print("\nComputed solution:")
print(x_computed_ill)

# Calculate the element-wise absolute errors
abs_errors = np.abs(x_true_ill - x_computed_ill)
print("\nAbsolute errors:")
print(abs_errors)

# Calculate the relative error
error_ill = norm(x_true_ill - x_computed_ill) / norm(x_true_ill)
print(f"\nRelative error: {error_ill:.8e}")

Let's visualize the true solution versus the computed solution:

In [ ]:
# Create a dataframe for comparison
df_comparison = pd.DataFrame({
    'Index': range(n),
    'True Solution': x_true_ill,
    'Computed Solution': x_computed_ill,
    'Absolute Error': abs_errors
})

print("Comparison of True vs Computed Solutions:")
df_comparison

Now, let's introduce a tiny perturbation to the right-hand side:

In [ ]:
# Add a small perturbation to b_ill
delta_ill = 1e-10
np.random.seed(123)  # Different seed to get more visible changes
perturbation = delta_ill * np.random.randn(n)
b_perturbed_ill = b_ill + perturbation

# Solve with the perturbed right-hand side
x_perturbed_ill = solve(A_ill, b_perturbed_ill)

# Calculate the relative changes
input_change_ill = norm(b_ill - b_perturbed_ill) / norm(b_ill)
output_change_ill = norm(x_computed_ill - x_perturbed_ill) / norm(x_computed_ill)

print(f"Relative change in input (b): {input_change_ill:.8e}")
print(f"Relative change in output (x): {output_change_ill:.8e}")
print(f"Ratio of output change to input change: {output_change_ill/input_change_ill:.4e}")

# Compare this ratio to the condition number
print(f"Condition number: {cond_number_ill:.4e}")

# Calculate errors of perturbed solution vs true solution
perturbed_error = norm(x_true_ill - x_perturbed_ill) / norm(x_true_ill)
print(f"\nRelative error of perturbed solution: {perturbed_error:.8e}")

For the ill-conditioned matrix, a tiny change in the input results in a much larger change in the output. The ratio of these changes is approximately the condition number.

In [ ]:
# Create a more detailed visualization
plt.figure(figsize=(14, 10))

# Plot 1: Bar chart comparing the solutions
plt.subplot(2, 1, 1)
indices = np.arange(n)
width = 0.25

bar1 = plt.bar(indices - width, x_true_ill, width, label='True Solution', color='green')
bar2 = plt.bar(indices, x_computed_ill, width, label='Computed Solution', color='blue')
bar3 = plt.bar(indices + width, x_perturbed_ill, width, label='Perturbed Solution', color='red')

plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Comparison of Solutions for Ill-conditioned System')
plt.xticks(indices)
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Errors (on a logarithmic scale)
plt.subplot(2, 1, 2)
errors_computed = np.abs(x_true_ill - x_computed_ill)
errors_perturbed = np.abs(x_true_ill - x_perturbed_ill)
diff_solutions = np.abs(x_computed_ill - x_perturbed_ill)

plt.semilogy(indices, errors_computed, 'o-', label='Error in Computed Solution', color='blue')
plt.semilogy(indices, errors_perturbed, 's-', label='Error in Perturbed Solution', color='red')
plt.semilogy(indices, diff_solutions, '^-', label='Difference Between Solutions', color='purple')

plt.xlabel('Index')
plt.ylabel('Absolute Error (log scale)')
plt.title('Errors in Solutions (Logarithmic Scale)')
plt.xticks(indices)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

The above visualization clearly shows how dramatically the solutions can differ from the true values in an ill-conditioned system, even when the system is solved with high precision and without explicit perturbations. The logarithmic plot of errors highlights that some components of the solution have very large errors.

Now, let's look at another extreme case using an even larger Hilbert matrix:

In [ ]:
# Create an even more ill-conditioned system
n_extreme = 13  # A much more ill-conditioned case
A_extreme = hilbert_matrix(n_extreme)
x_true_extreme = np.ones(n_extreme)  # Simple true solution
b_extreme = A_extreme @ x_true_extreme

# Calculate the condition number
cond_extreme = cond(A_extreme)
print(f"Condition number: {cond_extreme:.4e}")

# Expected digits of accuracy with double precision (52 bits of mantissa)
digits_double = 15  # Roughly 15-16 decimal digits for IEEE 754 double precision
expected_accurate_digits = max(0, digits_double - np.log10(cond_extreme))
print(f"Expected accurate digits: {expected_accurate_digits:.1f}")

# Solve the system
x_computed_extreme = solve(A_extreme, b_extreme)

# Calculate relative error
error_extreme = norm(x_true_extreme - x_computed_extreme) / norm(x_true_extreme)
print(f"Relative error: {error_extreme:.8e}")

# Visualize the results
plt.figure(figsize=(12, 6))
plt.semilogy(range(n_extreme), np.abs(x_true_extreme - x_computed_extreme), 'o-', linewidth=2)
plt.grid(True)
plt.xlabel('Index')
plt.ylabel('Absolute Error (log scale)')
plt.title(f'Errors in Solution with Condition Number {cond_extreme:.1e}')
plt.xticks(range(n_extreme))
plt.tight_layout()
plt.show()

## Condition Number vs. Matrix Size for Hilbert Matrices

Let's examine how the condition number of the Hilbert matrix grows with its size:

In [ ]:
# Calculate condition numbers for Hilbert matrices of different sizes
sizes = range(2, 16)
condition_numbers = [cond(hilbert_matrix(i)) for i in sizes]

# Plot the condition number vs matrix size
plt.figure(figsize=(10, 6))
plt.semilogy(sizes, condition_numbers, 'o-', linewidth=2)
plt.grid(True)
plt.xlabel('Matrix Size')
plt.ylabel('Condition Number (log scale)')
plt.title('Condition Number vs. Matrix Size for Hilbert Matrices')
plt.xticks(sizes)
plt.tight_layout()
plt.show()

# Create a table of condition numbers and expected accurate digits
digits_double = 15  # Roughly 15-16 decimal digits for IEEE 754 double precision
expected_accurate_digits = [max(0, digits_double - np.log10(kappa)) for kappa in condition_numbers]

cond_table = pd.DataFrame({
    'Matrix Size': list(sizes),
    'Condition Number': condition_numbers,
    'Expected Accurate Digits': expected_accurate_digits
})
cond_table

The condition number of the Hilbert matrix grows exponentially with its size. For larger sizes, the matrix becomes so ill-conditioned that numerical solutions become essentially meaningless. Note how the expected accurate digits drop to zero for larger matrices!

## How to Check Condition Number

In NumPy, you can check the condition number of a matrix using the `numpy.linalg.cond` function. Let's look at how to use it and interpret the results:

In [ ]:
# Create matrices with different condition numbers
A1 = np.array([[1, 0], [0, 1]])  # Identity matrix: perfectly conditioned
A2 = np.array([[10, 1], [1, 10]])  # Well-conditioned
A3 = np.array([[1, 0.999], [0.999, 1]])  # Ill-conditioned
A4 = hilbert_matrix(5)  # Very ill-conditioned

# Calculate condition numbers
cond_A1 = cond(A1)
cond_A2 = cond(A2)
cond_A3 = cond(A3)
cond_A4 = cond(A4)

print(f"Condition number of identity matrix: {cond_A1:.4f}")
print(f"Condition number of well-conditioned matrix: {cond_A2:.4f}")
print(f"Condition number of ill-conditioned matrix: {cond_A3:.4e}")
print(f"Condition number of Hilbert matrix: {cond_A4:.4e}")

# Guidelines for interpreting condition numbers
print("\nInterpreting condition numbers:")
print("κ ≈ 1: Well-conditioned")
print("1 < κ < 1000: Moderately well-conditioned")
print("1000 ≤ κ < 10^6: Ill-conditioned")
print("κ ≥ 10^6: Very ill-conditioned")

## Techniques to Improve Numerical Stability

When dealing with ill-conditioned matrices, several techniques can help improve numerical stability:

1. **Pivoting strategies** in Gaussian elimination
2. **Scaling** the matrix and right-hand side
3. Using **regularization** techniques
4. Employing more stable factorization methods like **SVD**

Let's implement a simple scaling approach and see how it affects the condition number:

In [ ]:
# Create an ill-conditioned matrix with widely varying magnitudes
A_unscaled = np.array([
    [1e5, 2e5],
    [3, 4]
])

print("Unscaled matrix:")
print(A_unscaled)
print(f"Condition number: {cond(A_unscaled):.4e}")

# Scale rows to have similar magnitudes
row_norms = np.array([norm(A_unscaled[i, :]) for i in range(A_unscaled.shape[0])])
D = np.diag(1.0 / row_norms)
A_scaled = D @ A_unscaled

print("\nScaled matrix:")
print(A_scaled)
print(f"Condition number: {cond(A_scaled):.4e}")

## Rule of Thumb: Digits of Accuracy

A practical rule of thumb: If a matrix has condition number $\kappa(A) = 10^k$, you can expect to lose up to $k$ digits of accuracy in the solution due to roundoff error.

For example, with a condition number of $10^6$, you might lose about 6 digits of accuracy in your solution.

For IEEE 754 double precision (which NumPy uses by default), you have about 15-16 decimal digits of precision. So if your condition number is $10^{12}$, you might have only 3-4 accurate digits in your solution, and if it's $10^{16}$ or larger, you can't expect any accurate digits!

## Practical Guidelines for Dealing with Ill-Conditioned Systems

1. **Always calculate the condition number** of the coefficient matrix before solving a linear system.

2. **Be cautious of results** when the condition number is high (> 10^6).

3. **Use higher precision** when possible for ill-conditioned problems.

4. **Consider scaling** the rows or columns of the matrix to improve conditioning.

5. **Use stable algorithms** like LU decomposition with pivoting or SVD for solving ill-conditioned systems.

6. **Reformulate the problem** if possible to avoid ill-conditioning.

## Exercises for Students

1. **Create and analyze a matrix** of your own design that is ill-conditioned.
2. **Modify the matrix** to improve its condition number.
3. **Investigate how small perturbations** to an ill-conditioned system affect its solution for a problem of your choice.
4. **Apply the rule of thumb** to predict the number of accurate digits in a solution, and then verify your prediction.